# 🤖 AI-Powered FinOps Analysis with Gemini 2.5 Pro
\nThis notebook demonstrates how to use the **Gemini 2.5 Pro** model to perform a deep, interactive analysis of your cloud cost and usage data. It showcases advanced features like **Code Execution** and **URL Context**.

## ⚠️ Important: Setup and Authentication
\nBefore you begin, ensure you have completed the following steps:
\n1.  **Run the Setup Script**: From your terminal, in the project's root directory, run `bash scripts/setup_gcp_notebook.sh`.
2.  **Authenticate to Google Cloud**: Run `gcloud auth application-default login` in your terminal and follow the prompts.
3.  **Set Project ID**: Make sure your Google Cloud Project ID is set correctly in the `config.yaml` file.
4.  **Enable Vertex AI API**: Ensure the Vertex AI API is enabled in your Google Cloud project.

## 📚 Step 1: Imports and Initialization

In [ ]:
import logging
import pandas as pd
from IPython.display import Markdown, display
from finops_analysis_platform.config_manager import ConfigManager
from finops_analysis_platform.data_loader import GCSDataLoader
from finops_analysis_platform.gemini_service import initialize_gemini, generate_content
from google.genai.types import Tool, ToolCodeExecution, UrlContext

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

config_manager = ConfigManager(config_path='config.yaml')
project_id = config_manager.get('gcp.project_id')
location = config_manager.get('gcp.location', 'us-central1')

client = initialize_gemini(project_id=project_id, location=location)
model_id = 'gemini-2.5-pro'

print(f"✅ Gemini client initialized for project '{project_id}' using model '{model_id}'.")

## 📂 Step 2: Load Your Data
\nWe'll load the billing and recommendations data from your GCS bucket. If you're not authenticated, this will fall back to sample data.

In [ ]:
loader = GCSDataLoader(bucket_name=config_manager.get('gcs.bucket_name'))
data = loader.load_all_data()
billing_df = data.get('billing')
recommendations_df = data.get('recommendations')

print("✅ Data loaded successfully.")

## 💻 Step 3: Enhanced Code Execution
\nGemini can generate and execute Python code to answer questions about your data. This example first identifies the top spending services and then generates a visualization.

In [ ]:
code_execution_tool = Tool(code_execution=ToolCodeExecution())

prompt = """
Analyze the billing data provided in the pandas DataFrame `billing_df`.
1. Identify the top 5 most expensive services.
2. Calculate the total cost for each of these top 5 services.
3. Generate a bar chart to visualize the costs of these top 5 services and save it as `top_services_cost.png`.
"""

response = generate_content(client, model_id, prompt, tools=[code_execution_tool])

for part in response.candidates[0].content.parts:
    if part.executable_code:
        display(Markdown(f"**Generated Code:**\n```python\n{part.executable_code.code}\n```"))
    if part.code_execution_result:
        display(Markdown(f"**Execution Result:**\n```\n{part.code_execution_result.output}\n```"))

## 🔗 Step 4: Analysis with URL Context
\nGemini can use the content of URLs to provide more informed responses. Here, we'll ask it a question about Google Cloud's pricing, using a public documentation page as context.

In [ ]:
url_context_tool = Tool(url_context=UrlContext())

prompt = """
Based on the content of this URL: https://cloud.google.com/compute/all-pricing, what are the key differences in pricing between N2, E2, and C2 machine types? Summarize the main characteristics of each family.
"""

response = generate_content(client, model_id, prompt, tools=[url_context_tool])

if response.text:
    display(Markdown(response.text))

if response.candidates[0].url_context_metadata:
    print("\n--- URL Context Metadata ---")
    print(response.candidates[0].url_context_metadata)

## 💡 Step 5: Automated Insight Generation
\nFinally, we can use Gemini to synthesize our findings into a high-level summary for an executive audience.

In [ ]:
insights_prompt = """
Based on the analysis performed, we have identified the top spending services and have access to billing and recommendation data.
Please generate a short, actionable summary for a CFO, highlighting the key findings and recommending the next steps for cost optimization.
Assume the analysis revealed that 'Compute Engine' is the highest cost driver and that there are significant savings opportunities in 'Rightsizing' and 'Deleting Unattached Disks'.
"""

summary_response = generate_content(client, model_id, insights_prompt, tools=[]) # No tools needed for this summarization

if summary_response.text:
    display(Markdown(summary_response.text))